In [ ]:
from attalos.imgtxt_algorithms.main import *

In [ ]:
# Temp object using duck typing to replace command line arguments
args = lambda: None

args.image_feature_file_train = ""
args.text_feature_file_train = ""
args.image_feature_file_test = ""
args.text_feature_file_test = ""

args.word_vector_file = ""
args.word_vector_type = "glove" # "w2v"

args.model_type = "negsamp" #"wdv" #"naivesum" #"multihot"
args.cross_eval = False
args.in_memory = True
args.model_input_path = None
args.model_output_path = None
args.num_epochs = 500
args.batch_size = 100
args.learning_rate = 0.0001
args.verbose_eval = True
args.epoch_verbosity = 10

In [ ]:
logger.info("Parsing train and test datasets.")
train_dataset = Dataset(args.image_feature_file_train, args.text_feature_file_train, load_image_feats_in_mem=args.in_memory)
test_dataset = Dataset(args.image_feature_file_test, args.text_feature_file_test)

In [ ]:
logger.info("Reading word vectors from file.")
wv_model = load_wv_model(args.word_vector_file, args.word_vector_type)

In [ ]:
#sess.close()

In [ ]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.InteractiveSession(config=config)

In [ ]:
model_cls = ModelTypes[args.model_type].value
logger.info("Selecting model class: %s" % model_cls.__name__)
datasets = [train_dataset] if args.cross_eval else [train_dataset, test_dataset]
model = model_cls(wv_model, datasets, **vars(args))
model.initialize_model(sess)

In [ ]:
logger.info("Preparing test_dataset.")
fetches, feed_dict, truth = model.prep_predict(test_dataset)

In [ ]:
logger.info("Starting training phase.")
mid_eval = (fetches, feed_dict, truth) if args.verbose_eval else None
train(sess, model, args.num_epochs, train_dataset, args.batch_size, args.epoch_verbosity, mid_eval=mid_eval) #, train_dataset, wv_model, test_dataset=test_dataset, epoch_verbosity_rate=100)

In [ ]:
logger.info("Starting evaluation phase.")
evaluate(sess, model, fetches, feed_dict, truth)